In [ ]:
!cd ..; python3 setup.py develop

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from fastpathplanning.boxes import Box, BoxCollection
from fastpathplanning.polygonal import build_min_distance
from fastpathplanning.smooth import optimize_bezier_with_retiming

# Preprocessing

In [ ]:
d = 2
n = 5
sides = [2, .5]

boxes = [
    Box([   6.25,  4], [ 7.25,    5.75]),
    Box([ 5.5,  2.5], [ 7.5, 4.75]),
    Box([   1,    4], [   6,    5]),
    Box([   .25, 1.5], [7,    3]),
    Box([   2.25,    .75], [ 3, 2.5]),
    Box([  .5,  -.25], [ 1.5,  4.5]),
    Box([   2, 0], [   6.25,    1]),
    Box([ 4.75,  -.25], [   6, 3.75]),
    Box([ 0,  5.2], [   7, 6]),
]
B = BoxCollection(boxes)
G = B.line_graph()

In [ ]:
plt.figure()
B.plot2d(alpha=.5)
G.plot()

# Polygonal phase

In [ ]:
start = np.array([1, .25])
goal = np.array([.5, 5.6])

In [ ]:
# Solves all-to-one shortest path for a fixed goal location.
discrete_planner, runtime = G.shortest_path(goal)
runtime

In [ ]:
# Given start location finds box sequence (and initial trajectory).
box_seq, length, runtime = discrete_planner(start)
length, runtime

In [ ]:
# Builds optimization problem (can be done offline).
iterative = True
max_boxes = 2 * n
continuous_planner = build_min_distance(B, max_boxes, iterative, solver='ECOS')

# Refresh discrete plan.
box_seq, runtime, length = discrete_planner(start)

In [ ]:
box_seq, traj, length, iters, solver_time, runtime = continuous_planner(box_seq, start, goal)
length, solver_time, runtime

In [ ]:
plt.figure(figsize=(5, 5))
B.plot2d(alpha=.5)
B.plot2d(subset=box_seq, fc='None', ec='r')
plt.plot(*traj.T, c='b')

# Smooth phase

In [ ]:
# Fix box sequence.
L = np.array([B.boxes[i].l for i in box_seq])
U = np.array([B.boxes[i].u for i in box_seq])

# Cost coefficients.
alpha = {2: 1, 3: 1}

# Boundary conditions.
initial = {0: start}
final = {0: goal}

# Traversal time.
T = n

In [ ]:
# Initialize transition times.
durations = np.linalg.norm(traj[1:] - traj[:-1], axis=1)
durations *= T / sum(durations)

In [ ]:
path, sol_stats = optimize_bezier_with_retiming(L, U, durations, alpha, initial, final, verbose=True)

In [ ]:
plt.figure()
B.plot2d(alpha=.5)
path.plot(c='b')